In [32]:
!pip install transformers

     |████████████████████████████████| 2.1MB 11.8MB/s 
     |████████████████████████████████| 901kB 38.2MB/s 
     |████████████████████████████████| 3.3MB 74.7MB/s 


In [33]:
import re
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import collections
from nltk.metrics.scores import (precision, recall)
import pickle
import keras
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
# carga modelo tokenizador
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [4]:
# Esta funcion elimina links y nombres de usuarios
def remove_linksT(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    text = re.sub('\@\w+', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

## Cargamos los modelos entrenados

In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/NLP

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Copia de machine-learning_speech-recognition_7601-291468-0006.wav'
 modelos
 ModelosNuevos.ipynb
 noemoticon.csv
 private.py


Modelos Clasicos

In [6]:
file = open('mydrive/NLP/modelos/NaiveBayes.pickle', 'rb')
NB = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/BernoulliNB.pickle', 'rb')
BerNB = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/MNB.pickle', 'rb')
MultNB = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/LinearSVC.pickle', 'rb')
LinearSVC = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/SGDC.pickle', 'rb')
SGDC = pickle.load(file)
file.close()

# traemos el training y testing set
file = open('/mydrive/NLP/modelos/training.pickle', 'rb')
training_set = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/testing.pickle', 'rb')
testing_set = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/documents.pickle', 'rb')
documents = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/word_features.pickle', 'rb')
word_features = pickle.load(file)
file.close()

Modelos nuevos (RNN, BERT)

In [7]:
RNN1 = keras.models.load_model('/mydrive/NLP/modelos/model.h5')
RNN2 = keras.models.load_model('/mydrive/NLP/modelos/model0.h5')
RNN3 = keras.models.load_model('/mydrive/NLP/modelos/model2.h5')
#BERT = keras.models.load_model('/mydrive/NLP/modelos/modelBERT.h5')

# traemos el training y testing set
file = open('/mydrive/NLP/modelos/X_train.pickle', 'rb')
X_train = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/X_test.pickle', 'rb')
X_test = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/Y_train.pickle', 'rb')
Y_train = pickle.load(file)
file.close()

file = open('/mydrive/NLP/modelos/Y_test.pickle', 'rb')
Y_test = pickle.load(file)
file.close()

## Mostramos sus valores de acuracy, presition and recall

In [ ]:
refsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)

In [ ]:
def tset(modelo):
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(testing_set):
        observed = modelo.classify(feats)
        testsets[observed].add(i)
    return testsets

In [ ]:
def metricas(modelo):
    print("accuracy percent:", (nltk.classify.accuracy(modelo, testing_set))*100)
    print('pos precision:', precision(refsets['pos'], testsets['pos']))
    print('pos recall:', recall(refsets['pos'], testsets['pos']))
    print('neg precision:', precision(refsets['neg'], testsets['neg']))
    print('neg recall:', recall(refsets['neg'], testsets['neg']))

In [ ]:
print("NaiveBayes")
testsets = tset(NB)
metricas(NB)

NaiveBayes
accuracy percent: 72.875
pos precision: 0.7059882935614588
pos recall: 0.784
neg precision: 0.7571669477234402
neg recall: 0.6735


In [ ]:
print("Bernoulli NB")
testsets = tset(BerNB)
metricas(BerNB)

Bernoulli NB
accuracy percent: 73.125
pos precision: 0.7036547776309996
pos recall: 0.799
neg precision: 0.7674956622325043
neg recall: 0.6635


In [ ]:
print("Multinomial NB")
testsets = tset(MultNB)
metricas(MultNB)

Multinomial NB
accuracy percent: 72.425
pos precision: 0.7149017728797317
pos recall: 0.746
neg precision: 0.7344485101934135
neg recall: 0.7025


In [ ]:
print("LinearSVC")
testsets = tset(LinearSVC)
metricas(LinearSVC)

LinearSVC
accuracy percent: 72.675
pos precision: 0.7102457116365323
pos recall: 0.766
neg precision: 0.7460661964188823
neg recall: 0.6875


In [ ]:
print("SGDC")
testsets = tset(SGDC)
metricas(SGDC)

SGDC
accuracy percent: 73.425
pos precision: 0.7196436943272386
pos recall: 0.7675
neg precision: 0.7509373326191752
neg recall: 0.701


In [ ]:
y_test = []
for i in Y_test:
  if i[0] == 1:
    y_test.append(0)
  else:
    y_test.append(1)
y_test = np.array(y_test)

In [ ]:
y_pred1 = RNN1.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

print("RNN 1:")
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred , average="macro"))
print("Recall: ", recall_score(y_test, y_pred , average="macro"))

RNN 1:
Accuracy:  0.756625
Precision:  0.7578376253722533
Recall:  0.7567341492544515


In [ ]:
y_pred1 = RNN2.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

print("RNN 2:")
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred , average="macro"))
print("Recall: ", recall_score(y_test, y_pred , average="macro"))

RNN 2:
Accuracy:  0.65525
Precision:  0.6553280552460898
Recall:  0.6552057331105561


In [ ]:
y_pred1 = RNN3.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)

print("RNN 3:")
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred , average="macro"))
print("Recall: ", recall_score(y_test, y_pred , average="macro"))

RNN 3:
Accuracy:  0.752875
Precision:  0.7531515864277849
Recall:  0.7529264215353275


## Creamos clasificador 

In [8]:
# Este objeto es nuestro clasificador
from collections import Counter
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            c = Counter(votes)
            moda = c.most_common(1)[0][0]
        return moda

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [9]:
# Clasificador por votos
voted_classifier = VoteClassifier(MultNB, BerNB, SGDC, LinearSVC, NB)
print(f"Accurracy para voted Classifier: {nltk.classify.accuracy(voted_classifier, testing_set)*100}")

Accurracy para voted Classifier: 68.0


In [10]:
import gensim
from nltk.tokenize.treebank import TreebankWordDetokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))
        
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [11]:
temp = []
#Splitting pd.Series to list
data_to_list = [x[0] for x in documents]
labels = [x[1] for x in documents]

#data_to_list = train['selected_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(remove_linksT(data_to_list[i]))
data_words = list(sent_to_words(temp))
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [12]:
def agregar_palabra(palabra):
  datos = data
  datos.append(palabra)
  palabras = 29387
  max_len = 200
  tokenizer1 = Tokenizer(num_words=palabras)
  tokenizer1.fit_on_texts(datos)
  sequences1 = tokenizer1.texts_to_sequences(datos)
  tweet = pad_sequences(sequences1, maxlen=max_len)
  return (tweet[-1])

In [13]:
def RNN1predict (palabra):
  y_pred1 = RNN1.predict(agregar_palabra(palabra))
  y_pred = np.argmax(y_pred1, axis=1)
  return y_pred[0]

def RNN2predict (palabra):
  y_pred1 = RNN2.predict(agregar_palabra(palabra))
  y_pred = np.argmax(y_pred1, axis=1)
  return y_pred[0]

def RNN3predict (palabra):
  y_pred1 = RNN3.predict(agregar_palabra(palabra))
  y_pred = np.argmax(y_pred1, axis=1)
  return y_pred[0]

In [14]:
def ClasificadorVotos(palabra):
  votos = []
  votos.append(RNN1predict(palabra))
  votos.append(RNN2predict(palabra))
  votos.append(RNN3predict(palabra))
  for i in range(len(votos)):
    if votos[i] == 0:
      votos[i] = 'neg'
    else:
      votos[i] = 'pos'
  votos.append(NB.classify(find_features(palabra)))
  votos.append(BerNB.classify(find_features(palabra)))
  votos.append(MultNB.classify(find_features(palabra)))
  votos.append(SGDC.classify(find_features(palabra)))
  return mode(votos)

## 2. Convertir audio a texto

In [15]:
# Instalar librerias que no se encuentran en colab
!pip3 install SpeechRecognition pydub

In [16]:
# importa las librerias
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
# crea objeto de reconocimiento de audio
r = sr.Recognizer()

# funcion que separa el audio y aplica reconociemiento de audio a cada parte
def get_large_audio_transcription(path):
    """
    funcion que separa el audio
    y aplica reconociemiento de audio a cada parte
    """
    # abre el archivo de audio usando pydub
    sound = AudioSegment.from_wav(path)  
    # divide el sonido cuando el silencio es de 700 milisegundos a mas
    chunks = split_on_silence(sound,
        # parámetro
        min_silence_len = 500,
        # ajustar
        silence_thresh = sound.dBFS-14,
        # mantener silencio por 1 segundo, ajustable
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # procesa cada parte 
    for i, audio_chunk in enumerate(chunks, start=1):
        # exporta el audio y lo guarda en
        # el directorio `folder_name`.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
 
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # convierte el segmento de audio en texto
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                #print(chunk_filename, ":", text)
                whole_text += text
    # Regresa el texto completo
    return whole_text

In [17]:
# Carga el archivo a convertir
import requests
import urllib.request
URL = 'https://github.com/x4nth055/pythoncode-tutorials/blob/master/machine-learning/speech-recognition/7601-291468-0006.wav?raw=true'

urllib.request.urlretrieve(URL, 'machine-learning_speech-recognition_7601-291468-0006.wav')

path = "machine-learning_speech-recognition_7601-291468-0006.wav"
transcripcion = get_large_audio_transcription(path)
print("\nFull text:", transcripcion)


Full text: His abode which he had fixed in a bowery or countryside. Had a short distance from the city. Just that one is now called dutch street. Sooner bounded with proofs of his ingenuity. Patterned smoke chair. That require the horse to work them. Dutch ovens that roasted meat without fire. Cards that went before the horses. Weather cox that turned against the wind another one headed contrivance. The astonished and confounded or beholders. 


In [18]:
voted_classifier.classify(find_features(transcripcion))

'neg'

## Tweets de usuario

In [19]:
import sys
import tweepy
import datetime, time 
!cp /mydrive/NLP/private.py ../
import private
import json

auth = tweepy.OAuthHandler(private.twitter_app_key, private.twitter_app_secret)
auth.set_access_token(private.twitter_key, private.twitter_secret)
api = tweepy.API(auth)
#parser=tweepy.parsers.JSONParser()

def get_tweets(api, username):
    page = 1
    tweetsFinal = []
    while page < 2:
      tweets = api.user_timeline(username, page = page)
      for tweet in tweets:
        obj = json.dumps(tweet._json['text'])
        tweetsFinal.append(str(obj))
        #json.loads(tweet._json)
        #tweetsFinal.append(str(tweet['text']))
      page += 1
    return tweetsFinal

In [31]:
tweets = get_tweets(api, 'InternetHippo')
print(tweets)
print(type(tweets))

['"almost\\u2026 https://t.co/8TsWJpram9"', '"https://t.co/xHvKsASgku"', '"You sound like an amateur asking someone to \\u201chang out\\u201d without a plan. Always include a full powerpoint with your p\\u2026 https://t.co/uXXEf7MjBn"', '"@Sturgeons_Law when you only want to defund the tax police"', '"@randygdub aw shit it\\u2019s true"', '"Every full moon elon musk transforms into a giant mole and digs those tunnels and he had to build a whole company a\\u2026 https://t.co/Fjt0g5FTu6"', '"RT @ameliaelizalde: the fact that exercise works but only if you do it all the time.....toxic"', '"they don\\u2019t make presidents like this anymore rip https://t.co/2lamBeqlxo"', '"@DLNHLGSN big day for me"', '"@Lubchansky got \\u2018pandemic ass\\u2019 ? you\\u2019re not alone"', '"@fordm apartments are twice as  good now"', '"@ActNormalOrElse doing the right thing as a last resort since 1776"', '"migrant children at the border are being given copies of karl marx\\u2019s book \\u201chow to vote fo

In [38]:
tweets = get_tweets(api, 'InternetHippo')

for tweet in tweets:
  text = str(tweet)
  text = remove_linksT(text)
  agregar_palabra(text)
  print(ClasificadorVotos(text),text)

pos  almost 
pos  
neg  You sound like an amateur asking someone to chang out d without plan Always include full powerpoint with your u 
neg  when you only want to defund the tax police 
pos  aw shit it s true 
pos  Every full moon elon musk transforms into giant mole and digs those tunnels and he had to build whole company u 
neg  RT the fact that exercise works but only if you do it all the time toxic 
neg  they don t make presidents like this anymore rip 
pos  big day for me 
pos  got pandemic ass you re not alone 
pos  apartments are twice as good now 
pos  doing the right thing as last resort since 
pos  migrant children at the border are being given copies of karl marx s book chow to vote for democrats illegally d ame 
pos  associate director of wife relations 
pos  for widows 
pos  Seems wrong that wife is always called wife without regard for duration of wifehood There should be ranks based 
pos  help me help you 
pos  the reactions are so funny actually think the government sh

### Tweets por usuario con modelo Bert

In [36]:
modelBERT = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
modelBERT.load_weights('/mydrive/NLP/modelos/bert_weights.h5')

# imprime resumen del modelo
modelBERT.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_113 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [39]:
tweets = get_tweets(api, 'InternetHippo')
tweet2 = []
for tweet in tweets:
  text = str(tweet)
  text = remove_linksT(text)
  tweet2.append(text)

tf_batch = tokenizer(tweet2, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = modelBERT(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(tweet2)):
  print(tweet2[i], ": \n", labels[label[i]])

 almost  : 
 Negative
  : 
 Negative
 You sound like an amateur asking someone to chang out d without plan Always include full powerpoint with your u  : 
 Negative
 when you only want to defund the tax police  : 
 Negative
 aw shit it s true  : 
 Negative
 Every full moon elon musk transforms into giant mole and digs those tunnels and he had to build whole company u  : 
 Negative
 RT the fact that exercise works but only if you do it all the time toxic  : 
 Positive
 they don t make presidents like this anymore rip  : 
 Negative
 big day for me  : 
 Positive
 got pandemic ass you re not alone  : 
 Negative
 apartments are twice as good now  : 
 Positive
 doing the right thing as last resort since  : 
 Negative
 migrant children at the border are being given copies of karl marx s book chow to vote for democrats illegally d ame  : 
 Negative
 associate director of wife relations  : 
 Positive
 for widows  : 
 Negative
 Seems wrong that wife is always called wife without regard for durati

## 4. Buscar los post y coments de un redditor

In [23]:
! pip install praw

     |████████████████████████████████| 163kB 12.5MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 


In [24]:
import praw
import re
reddit = praw.Reddit(client_id='ZRHe0FYX80KmBg', 
                     client_secret='n8NSRCdkGHnVn0E_WC16mqrnGA7kpA', 
                     user_agent='robertonh62', 
                     username='robertonh62', 
                     password='NLP123456')

In [25]:
def remove_linksR(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    text = re.sub('(\s)r/\w+', ' ', text)
    text = re.sub('(\s)u/\w+', ' ', text)
    return text

In [26]:
def get_post(username):
    post = []
    for submission in reddit.redditor(username).submissions.top("all"):
        text = submission.selftext
        text = remove_linksR(text)
        post.append(text)
    return post

In [27]:
def get_coments(username):
    coments = []
    for comment in reddit.redditor(username).comments.new(limit=50):
        text = comment.body
        text = remove_linksR(text)
        coments.append(text)
    return coments

In [29]:
posts = get_post('league')
print(posts)
for post in posts:
  print(ClasificadorVotos(post), post)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



[]
